## CVM Oferta Públicas de Distribuição

Limpeza realizada por Ricardo Dahis  
Dados originais obtidos em [Dados Abertos CVM](http://dados.cvm.gov.br/dataset/oferta-distrib)   
E os dados processados se encontram em [Base dos Dados](https://basedosdados.org/dataset/br-cvm-oferta-publica-distribuicao)

### Prefácio

In [ ]:
import os
import shutil
from pathlib import Path
from typing import Union

import requests
import numpy as np
import pandas as pd
import basedosdados as bd

In [ ]:
folder = "/tmp/basedosdados"
url = "http://dados.cvm.gov.br/dados/OFERTA/DISTRIB/DADOS/oferta_distribuicao.csv"

### Download

In [ ]:
filepath = f"{folder}/data.zip"
os.makedirs(folder, exist_ok=True)
with open(filepath, "wb") as file:
    response = requests.get(url)
    file.write(response.content)
    shutil.unpack_archive(filepath)

### Limpeza

In [ ]:
#############################
# Dia
# dia.csv
#############################

dia_in_filepath = f"{folder}/dia.csv"
dia_ou_filepath = f"{folder}/bd_dia.csv"

df = pd.read_csv(
    dia_in_filepath,
    sep=";",
    keep_default_na=False,
    encoding="latin1",
    dtype=object
)

df.columns = [
    "numero_processo",
    "numero_registro_oferta",
    "tipo_oferta",
    "tipo_componente_oferta_mista",
    "tipo_ativo",
    "cnpj_emissor",
    "nome_emissor",
    "cnpj_lider",
    "nome_lider",
    "nome_vendedor",
    "cnpj_ofertante",
    "nome_ofertante",
    "rito_oferta",
    "modalidade_registro_oferta",
    "modalidade_dispensa_oferta",
    "data_abertura_processo",
    "data_protocolo",
    "data_dispensa_oferta",
    "data_registro_oferta",
    "data_encerramento_oferta",
    "emissao",
    "classe_ativo",
    "serie",
    "especie_ativo",
    "forma_ativo",
    "data_emissao",
    "data_vencimento",
    "quantidade_sem_lote_suplementar",
    "quantidade_no_lote_suplementar",
    "quantidade_total",
    "preco_unitario",
    "valor_total",
    "oferta_inicial",
    "oferta_incentivo_fiscal",
    "oferta_regime_fiduciario",
    "atualizacao_monetaria",
    "juros",
    "projeto_audiovisual",
]

df.loc[(df["oferta_inicial"] =="N"), "oferta_inicial"] = "Não"
df.loc[(df["oferta_inicial"] =="S"), "oferta_inicial"] = "Sim"

df.loc[(df["oferta_incentivo_fiscal"] =="N"), "oferta_incentivo_fiscal"] = "Não"
df.loc[(df["oferta_incentivo_fiscal"] =="S"), "oferta_incentivo_fiscal"] = "Sim"

df.loc[(df["oferta_regime_fiduciario"] =="N"), "oferta_regime_fiduciario"] = "Não"
df.loc[(df["oferta_regime_fiduciario"] =="S"), "oferta_regime_fiduciario"] = "Sim"

df.to_csv(dia_ou_filepath, index=False)

### Upload
Adaptado do código em `basedosdados_template` por Diego Oliveira

In [ ]:
def upload_to_gcs(dataset_id: str, table_id: str, path: Union[str, Path]) -> None:
    """Upload a bunch of CSVs to Google Cloud Storage using basedosdados library"""
    tb = bd.Table(dataset_id=dataset_id, table_id=table_id)
    st = bd.Storage(dataset_id=dataset_id, table_id=table_id)

    if tb.table_exists(mode="staging"):
        tb.append(
            filepath=path,
            if_exists="replace",
        )

        log((f"Successfully uploaded {path} to "
             f"{tb.bucket_name}.staging.{dataset_id}.{table_id}"))
    else:
        log(("Table does not exist in STAGING, need to create it in local first.\n"
             "Create and publish the table in BigQuery first."))

In [ ]:
upload_to_gcs("br_cvm_oferta_publica_distribuicao", "dia", dia_ou_filepath)

### Epílogo

In [ ]:
shutil.rmtree(folder)